In [1]:
import os, os.path
import numpy as np
from scipy.special import sph_harm
import weave
from weave import converters
from numba import jit
%load_ext Cython

In [2]:
from colloids import boo, particles, periodic

In [3]:
path = '/data/mleocmach/Documents/Tsurusawa/0_Data_retracked/3_DenseGel/163A/2_ageing/'
pos = np.loadtxt(os.path.join(path, '163A_1415_ageing_t000.dat'), skiprows=2)
bonds = np.loadtxt(os.path.join(path, '163A_1415_ageing_t000.bonds'), dtype=int)
inside = np.min((pos-pos.min(0)>14) & (pos.max()-pos>14), -1)
ngbs = particles.bonds2ngbs(bonds, len(pos))

In [13]:
%timeit boo.weave_qlm(pos, ngbs, inside)

1 loop, best of 3: 305 ms per loop


In [4]:
%timeit boo.bonds2qlm(pos, bonds)

1 loops, best of 3: 517 ms per loop


In [22]:
import math
def python_qlm(pos, ngbs, inside, l=6):
    qlm = np.zeros([len(pos), l+1], np.complex128)
    cart = np.zeros(3)
    sph = np.zeros(3)
    for i in range(ngbs.shape[0]):
        if not inside[i]:
            continue
        nb = 0
        for j in range(ngbs.shape[1]):
            q = ngbs[i,j]
            if q < 0 or q >= len(pos):
                continue
            nb += 1
            cart[:] = pos[i] - pos[q]
            sph[0] = cart[0]*cart[0] + cart[1]*cart[1] + cart[2]*cart[2]
            if cart[2]**2 == sph[0] or sph[0]+1.0 == 1.0:
                sph[1:] = 0
            else:
                sph[0] = math.sqrt(sph[0])
                sph[1] = math.acos(cart[2]/sph[0])
                sph[2] = math.atan2(cart[1], cart[0])
                if sph[2] < 0:
                    sph[2] += 2.0*math.pi
            qlm[i] += sph_harm(
                np.arange(l+1), l, 
                sph[2], 
                sph[1]
                )
        if nb>0:
            qlm[i] /= nb
    return qlm

In [23]:
%timeit python_qlm(pos, ngbs, inside)

1 loop, best of 3: 4.28 s per loop


In [5]:
%%cython --annotate
import numpy as np
from scipy.special.cython_special cimport sph_harm
cimport numpy as np
from libc.math cimport sqrt, acos, atan2, M_PI
cimport cython

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True) # turn off division by zero checking for entire function
def cython_qlm(
    np.ndarray[np.float64_t, ndim=2] pos, 
    np.ndarray[np.int64_t, ndim=2] ngbs, 
    np.ndarray[np.uint8_t, ndim=1, cast=True] inside, 
    int l=6
):
    cdef int i,j, q, nb, m, d
    cdef double r, r2, theta, phi, z
    cdef long Npos = pos.shape[0]
    cdef long Nngb = ngbs.shape[1]
    cdef np.ndarray[dtype=np.complex128_t, ndim=2] qlm = np.zeros([len(pos), l+1], np.complex128)
    #cdef np.ndarray[dtype=double, ndim=1] cart = np.zeros(3, float)
    cdef double[3] cart
    for i in range(Npos):
        if not inside[i]:
            continue
        nb = 0
        for j in range(Nngb):
            q = ngbs[i,j]
            if q < 0 or q >= Npos:
                continue
            nb += 1
            for d in range(3):
                cart[d] = pos[i,d] - pos[q,d]
            r2 = cart[0]**2 + cart[1]**2 + cart[2]**2
            if cart[2]**2 == r2 or r2+1.0 == 1.0:
                theta = 0
                phi = 0
            else:
                r = sqrt(r2)
                z = cart[2]
                phi = acos(z/r)
                theta = atan2(cart[1], cart[0])
                if theta < 0:
                    theta += 2.0*M_PI
            for m in range(l+1):
                qlm[i,m] = qlm[i,m] + sph_harm(m, l, theta, phi)
        if nb>0:
            for m in range(l+1):
                qlm[i,m] = qlm[i,m] / nb
    return qlm

In file included from /home/mathieu/anaconda3/envs/python2/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1809:0,
                 from /home/mathieu/anaconda3/envs/python2/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /home/mathieu/anaconda3/envs/python2/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /data/mleocmach/.cache/ipython/cython/_cython_magic_dff929309038b07cc4cb4b3c6acc7e68.c:257:
/home/mathieu/anaconda3/envs/python2/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^
In file included from /home/mathieu/anaconda3/envs/python2/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:27:0,
                 from /home/mathieu/anaconda3/env

In [6]:
%timeit cython_qlm(pos, ngbs, inside)

1 loops, best of 3: 540 ms per loop


In [94]:
sph_harm?

In [134]:
qlm_w = boo.weave_qlm(pos, ngbs, inside)

In [201]:
qlm_c = cython_qlm(pos, ngbs, inside)

In [202]:
np.mean(np.abs(qlm_w - qlm_c)<1e-16)

0.81391235399957307

In [187]:
qlm_w[0]

array([ 0.04028486+0.j        ,  0.05799787+0.06639547j,
        0.04605032-0.09580622j, -0.05903408-0.0054351j ,
        0.13045725-0.20548051j, -0.18253487-0.07674777j,
        0.09087308+0.00445716j])

In [188]:
qlm_c[0]

array([ 0.04028486+0.j        ,  0.05799787+0.06639547j,
        0.04605032-0.09580622j, -0.05903408-0.0054351j ,
        0.13045725-0.20548051j, -0.18253487-0.07674777j,
        0.09087308+0.00445716j])

In [183]:
sph_harm(0, 6, 3.685749891321086, 1.4481876153314692)

(-0.22243770324198525+0j)

In [182]:
sph_harm?

# Periodicity

In [6]:
periodic.periodify(np.array([[0,0,0]]), np.array([[1,1,1]]))

array([[1, 1, 1]])

In [7]:
periodic.periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [3,3,3])

array([[1, 1, 1]])

In [8]:
periodic.periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [3,1,2])

array([[1, 0, 1]])

In [12]:
periodic.periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [0.9,0.5,0.5])

array([[ 0.1,  0. ,  0. ]])

In [10]:
reload(periodic)

<module 'colloids.periodic' from '/home/mathieu/src/colloids/python/colloids/periodic.py'>

In [54]:
u = np.zeros([1000000, 3])
v = 3*np.random.rand(1000000, 3)
u.shape, v.shape

((1000000, 3), (1000000, 3))

In [71]:
%timeit periodic.periodify(u, v, [0.9,0.5,0.5])

1 loop, best of 3: 371 ms per loop


In [26]:
def python_periodify(u, v, periods=None):
    assert u.shape == v.shape
    diff = np.array(v, float) - u
    if periods is None:
        return diff
    assert len(periods) == u.shape[-1]
    #ensures the largest coordinate is smaller than half a period
    half = 0.5*np.array(periods)
    for i in range(diff.shape[0]):
        for j in range(diff.shape[1]):
            while diff[i,j] > half[j]:
                diff[i,j] -= periods[j]
            while diff[i,j] < -half[j]:
                diff[i,j] += periods[j]
    return diff

In [27]:
python_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]))

array([[ 1.,  1.,  1.]])

In [28]:
python_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [3,3,3])

array([[ 1.,  1.,  1.]])

In [29]:
python_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [3,1,2])

array([[ 1.,  0.,  1.]])

In [30]:
python_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [0.9,0.5,0.5])

array([[ 0.1,  0. ,  0. ]])

In [31]:
%timeit python_periodify(u, v, [0.9,0.5,0.5])

1 loop, best of 3: 8.22 s per loop


In [41]:
@jit
def numba_periodify(u, v, periods=None):
    """Given two arrays of points in a d-dimentional space with periodic boundary conditions, find the shortest vector between each pair"""
    assert u.shape == v.shape
    diff = np.array(v, float) - u
    if periods is None:
        return diff
    assert len(periods) == u.shape[-1]
    #ensures the largest coordinate is smaller than half a period
    half = 0.5*np.array(periods)
    pers = np.tile(periods, [len(diff),1])
    toolarge = diff > half
    while np.any(toolarge):
        diff[toolarge] -= pers[toolarge]
        toolarge = diff > half
    #same for negative coordinates
    toosmall = diff < -half
    while np.any(toosmall):
        diff[toosmall] += pers[toosmall]
        toosmall = diff < -half
    return diff

In [42]:
numba_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]))

array([[ 1.,  1.,  1.]])

In [35]:
numba_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [3,3,3])

array([[ 1.,  1.,  1.]])

In [36]:
numba_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [3,1,2])

array([[ 1.,  0.,  1.]])

In [37]:
numba_periodify(np.array([[0,0,0]]), np.array([[1,1,1]]), [0.9,0.5,0.5])

array([[ 0.1,  0. ,  0. ]])

In [38]:
%timeit numba_periodify(u, v, [0.9,0.5,0.5])

1 loop, best of 3: 345 ms per loop


In [78]:
%%cython --annotate
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True) # turn off division by zero checking for entire function
def cython_periodify(
    np.ndarray[np.float64_t, ndim=2] u, 
    np.ndarray[np.float64_t, ndim=2] v, 
    periods=None
):
    assert u.shape[0] == v.shape[0]
    assert u.shape[1] == v.shape[1]
    #cdef np.ndarray[np.float64_t, ndim=2] diff = v - u
    if periods is None:
        return v - u
    assert len(periods) == u.shape[1]
    cdef np.ndarray[np.float64_t, ndim=2] diff = np.zeros_like(u)
    #ensures the largest coordinate is smaller than half a period
    cdef np.ndarray[np.float64_t, ndim=1] pers = np.array(periods)
    cdef np.ndarray[np.float64_t, ndim=1] half = 0.5*np.array(periods)
    cdef int i,j
    cdef float d,h,p
    for i in range(diff.shape[0]):
        for j in range(diff.shape[1]):
            d = v[i,j] - u[i,j]
            h = half[j]
            p = pers[j]
            while d > h:
                d -= p
            while d < -h:
                d += p
            diff[i,j] = d
    return diff

In file included from /home/mathieu/anaconda3/envs/py27/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1809:0,
                 from /home/mathieu/anaconda3/envs/py27/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /home/mathieu/anaconda3/envs/py27/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/mathieu/.cache/ipython/cython/_cython_magic_b0fa00b67fd674d7e8dd3f97299c9047.c:507:
/home/mathieu/anaconda3/envs/py27/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^


In [79]:
%timeit cython_periodify(u, v, np.array([0.9,0.5,0.5]))

10 loops, best of 3: 52.1 ms per loop


In [76]:
%timeit v-u

100 loops, best of 3: 8.74 ms per loop


In [80]:
%timeit cython_periodify(u, v)

100 loops, best of 3: 8.75 ms per loop


In [89]:
from colloids import periodify

ImportError: cannot import name periodify

In [85]:
periodify?

## $w_\ell$

$$ w_\ell = \sum_{m_1+m_2+m_3=0} 
			\left( \begin{array}{ccc}
				\ell & \ell & \ell \\
				m_1 & m_2 & m_3 
			\end{array} \right)
			q_{\ell m_1} q_{\ell m_2} q_{\ell m_3}
			$$

In [7]:
qlm = boo.bonds2qlm(pos, bonds)

In [9]:
%timeit boo.wl(qlm)

1 loops, best of 3: 482 ms per loop


In [21]:
from colloids.boo import get_qlm, get_w3j
def python_wl(qlm):
    l = qlm.shape[1]-1
    w = np.zeros(qlm.shape[0], qlm.dtype)
    for m1 in range(-l, l+1):
        for m2 in range(-l, l+1):
            m3 = -m1-m2
            if -l<=m3 and m3<=l:
                w+= get_w3j(l, [m1, m2, m3]) * get_qlm(qlm, m1) * get_qlm(qlm, m2) * get_qlm(qlm, m3)
    return w.real

In [31]:
%timeit python_wl(qlm)

10 loops, best of 3: 113 ms per loop


In [18]:
np.sum(boo.wl(qlm) - python_wl(qlm).real > 1e-12)

0

In [30]:
@jit
def numba_wl(qlm):
    l = qlm.shape[1]-1
    w = np.zeros(qlm.shape[0], qlm.dtype)
    for m1 in range(-l, l+1):
        for m2 in range(-l, l+1):
            m3 = -m1-m2
            if -l<=m3 and m3<=l:
                w+= get_w3j(l, [m1, m2, m3]) * get_qlm(qlm, m1) * get_qlm(qlm, m2) * get_qlm(qlm, m3)
    return w.real

In [33]:
%timeit numba_wl(qlm)

10 loops, best of 3: 111 ms per loop


## Spatial correlation

In [27]:
def coarsegrain_qlm(qlm, bonds, inside):
    """Coarse grain the bond orientational order on the neighbourhood of a particle
    $$Q_{\ell m}(i) = \frac{1}{N_i+1}\left( q_{\ell m}(i) +  \sum_{j=0}^{N_i} q_{\ell m}(j)\right)$$
    Returns Qlm and the mask of the valid particles
    """
    #Valid particles must be valid themselves have only valid neighbours
    inside2 = np.copy(inside)
    np.bitwise_and.at(inside2, bonds[:,0], inside[bonds[:,1]])
    np.bitwise_and.at(inside2, bonds[:,1], inside[bonds[:,0]])
    #number of neighbours
    Nngb = np.zeros(len(qlm), int)
    np.add.at(Nngb, bonds.ravel(), 1)
    #sum the boo coefficients of all the neighbours
    Qlm = np.zeros_like(qlm)
    np.add.at(Qlm, bonds[:,0], qlm[bonds[:,1]])
    np.add.at(Qlm, bonds[:,1], qlm[bonds[:,0]])
    Qlm[np.bitwise_not(inside2)] = 0
    return Qlm / np.maximum(1, Nngb)[:,None], inside2

In [37]:
Qlm, inside2 = coarsegrain_qlm(qlm, bonds, inside)

In [29]:
%timeit coarsegrain_qlm(qlm, bonds, inside)

1 loops, best of 3: 220 ms per loop


In [34]:
@jit
def numba_coarsegrain_qlm(qlm, bonds, inside):
    """Coarse grain the bond orientational order on the neighbourhood of a particle
    $$Q_{\ell m}(i) = \frac{1}{N_i+1}\left( q_{\ell m}(i) +  \sum_{j=0}^{N_i} q_{\ell m}(j)\right)$$
    Returns Qlm and the mask of the valid particles
    """
    #Valid particles must be valid themselves have only valid neighbours
    inside2 = np.copy(inside)
    np.bitwise_and.at(inside2, bonds[:,0], inside[bonds[:,1]])
    np.bitwise_and.at(inside2, bonds[:,1], inside[bonds[:,0]])
    #number of neighbours
    Nngb = np.zeros(len(qlm), int)
    np.add.at(Nngb, bonds.ravel(), 1)
    #sum the boo coefficients of all the neighbours
    Qlm = np.zeros_like(qlm)
    np.add.at(Qlm, bonds[:,0], qlm[bonds[:,1]])
    np.add.at(Qlm, bonds[:,1], qlm[bonds[:,0]])
    Qlm[np.bitwise_not(inside2)] = 0
    return Qlm / np.maximum(1, Nngb)[:,None], inside2

In [36]:
%timeit numba_coarsegrain_qlm(qlm, bonds, inside)

1 loops, best of 3: 227 ms per loop


In [39]:
%timeit boo.gG_l(pos, qlm, Qlm, inside2, Nbins=250, maxdist=30.0)

1 loops, best of 3: 1.04 s per loop


In [ ]:
%timeit boo.gG_l(pos, qlm, Qlm, inside2, Nbins=1000, maxdist=120.0)